# Benchmarks

## Initialize

In [ ]:
%load_ext autoreload
%autoreload 2

import os
from tqdm.auto import tqdm
import pathlib

import numpy as np
import pandas as pd
import lifelines
import pandas as pd

from joblib import Parallel, delayed
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")
import shutil

import plotly.express as px
import plotly.graph_objects as go
from plotly.graph_objects import Box

import matplotlib.pyplot as plt
from lifelines import CRCSplineFitter
import warnings
from lifelines.utils import CensoringType

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

from IPython.display import clear_output
import pathlib

from lifelines.utils import concordance_index

In [ ]:
node = !hostname
if "sc" in node[0]:
    base_path = "/sc-projects/sc-proj-ukb-cvd"
else: base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"
print(base_path)

project_name = "210714_metabolomics"
#data_path = "/data/analysis/ag-reils/steinfej"
data_pre = f"{base_path}/data/2_datasets_pre/{project_name}"
data_post = f"{base_path}/data/3_datasets_post/{project_name}"

project_label = "21_metabolomics_multitask"
project_path = f"{base_path}/results/projects/{project_label}"
figures_path = f"{project_path}/figures"
data_results_path = f"{project_path}/data"
pathlib.Path(figures_path).mkdir(parents=True, exist_ok=True)
pathlib.Path(data_results_path).mkdir(parents=True, exist_ok=True)

In [ ]:
endpoints = [
    #Cardiovascular
    'M_MACE',
    'M_coronary_heart_disease',
    'M_cerebral_stroke',
    'M_peripheral_arterial_disease',
    'M_atrial_fibrillation',
    'M_heart_failure',
    'M_abdominal_aortic_aneurysm',
    'M_venous_thrombosis',
    
    # General IM
    'M_type_2_diabetes',
    'M_liver_disease',
    'M_renal_disease',
    
    # Pulmological
    'M_asthma', 
    'M_chronic_obstructuve_pulmonary_disease',  
    
    # Psychiatric/Neurological
    'M_all_cause_dementia',
    'M_parkinsons_disease',   
    
    # Cancers
    "M_lung_cancer",
    "M_non_melanoma_skin_cancer",
    "M_colon_cancer",
    "M_rectal_cancer",
    "M_prostate_cancer",
    "M_breast_cancer",
    
    # Ophtalmological
    'M_cataracts', 
    'M_glaucoma',
    
    # Traumatology
    'M_fractures',
]

In [ ]:
partitions = [str(p) for p in range(0, 22)]
splits = ["train", "valid", "test"]

In [ ]:
data_all = pd.read_feather(f"{data_results_path}/data_all_COX_211007_metabolomics.feather")

# Train COX

In [ ]:
modules = ["COX"]

In [ ]:
partitions=data_all.partition.unique().tolist()

In [ ]:
run="220126"
version=f"COX_{run}"
dump_path = f"{data_post}/{version}"
pathlib.Path(dump_path).mkdir(parents=True, exist_ok=True)

# General ablation - coef

In [ ]:
import pickle
d_list_general = []
for endpoint in tqdm(endpoints):
    eids = eids_dict[endpoint]
    data_endpoint = data_test_all.query("eid==@eids")
    for partition in partitions[:1]:
        data_partition = data_endpoint.query("partition==@partition")
        cph = pickle.load(open(f"{dump_path}/COX_{endpoint}_PANEL_{partition}.p", "rb"))
        cindex = cph.score(data_partition, scoring_method="concordance_index")
        d_list_general.append({"endpoint": endpoint, "cindex": cindex, "cph": cph})

In [ ]:
def absmean(x): return round(x.abs().mean(), 3)

df_rank = pd.DataFrame().append(d_list_general)
abl_df = pd.concat([cph.summary for cph in df_rank.cph.to_list()], axis=0).reset_index()
abl_name = abl_df.groupby("covariate")["coef", "p"].agg(absmean).sort_values("coef", ascending=False)#.iloc[-1]#.name
abl_name[["coef"]]